# Creating Analysis Variables for OPM dataset

In [1]:
# importing modules
import pandas as pd
import numpy as np
import os
import janitor

In [2]:
# loading in data
path = r'/Users/johntam/cf_data_analysis/tasks/A4/code/clean_data'
opm = pd.read_pickle(os.path.join(path,'opm.pkl'))

In [3]:
#  cleaning extreme price values
opm.loc[opm['prices'] > 100, 'prices'] = np.nan

In [3]:
# finding mean order_number by department groups
(opm
.groupby('department_id')
.agg({'order_number':['mean']})
)

,order_number
,mean
department_id,
1,15.457687
2,17.277920
3,17.179756
4,17.811403
5,15.215751
6,16.439806
7,17.225773
8,15.340520


compared to the result in the excercise, with this result we now can see all of the department id's. In addition, on average, the averages are higher than those produced with the subset. 

I'm not entirely sure this measure is informative? Given that the order_number just states which order number sequentially it is for a specific user. I believe we are not gaining any measure of quantity, but rather a measure of sequentiality.

In [9]:
opm['max_order'] = (opm
                    .groupby(['user_id'])['order_number']
                    .transform(np.max))

opm.loc[opm['max_order'] > 40, 'loyalty_flag'] = 'loyal customer'
opm.loc[(opm['max_order'] <= 40) & (opm['max_order'] > 10), 'loyalty_flag'] = 'regular customer'
opm.loc[opm['max_order'] <= 10, 'loyalty_flag'] = 'new customer'

opm[['user_id','max_order','loyalty_flag']].tail(10)

,user_id,max_order,loyalty_flag
32434202,202557,31,regular customer
32434203,202557,31,regular customer
32434204,202557,31,regular customer
32434205,202557,31,regular customer
32434206,202557,31,regular customer
32434207,202557,31,regular customer
32434208,202557,31,regular customer
32434209,203436,3,new customer
32434210,203436,3,new customer
32434211,205420,16,regular customer


In [28]:
opm.tail(20)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,prices,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spender_type,median_day_since_ordered,customer_freq
32433010,461844,200351,5,2,11,9.0,15408,2,0,Training Fluoride Free Toothpaste Apple & Ban...,...,7.0,mid-range product,regular days,most orders,11,regular customer,7.863636,low spender,19.0,regular customer
32433011,216145,205587,1,3,14,NaN,15408,6,0,Training Fluoride Free Toothpaste Apple & Ban...,...,7.0,mid-range product,slowest days,most orders,10,new customer,7.946970,low spender,30.0,non-frequent customer
32433012,2941852,200463,2,1,15,30.0,8682,11,0,All Purpose Eraser,...,8.8,mid-range product,busiest days,most orders,4,new customer,8.444737,low spender,30.0,non-frequent customer
32433013,2321285,203389,16,2,10,6.0,8682,7,0,All Purpose Eraser,...,8.8,mid-range product,regular days,most orders,17,regular customer,8.606030,low spender,5.0,frequent customer
32433014,609055,202557,1,0,13,NaN,43553,2,0,Orange Energy Shots,...,3.7,low-range product,busiest days,most orders,31,regular customer,6.905655,low spender,8.0,frequent customer
32433015,1928449,202557,4,4,9,16.0,43553,4,1,Orange Energy Shots,...,3.7,low-range product,slowest days,most orders,31,regular customer,6.905655,low spender,8.0,frequent customer
32433016,2222639,202557,6,5,9,4.0,43553,3,1,Orange Energy Shots,...,3.7,low-range product,regular days,most orders,31,regular customer,6.905655,low spender,8.0,frequent customer
32433017,339942,202557,8,4,10,4.0,43553,3,1,Orange Energy Shots,...,3.7,low-range product,slowest days,most orders,31,regular customer,6.905655,low spender,8.0,frequent customer
32433018,2798193,202557,9,4,13,7.0,43553,5,1,Orange Energy Shots,...,3.7,low-range product,slowest days,most orders,31,regular customer,6.905655,low spender,8.0,frequent customer
32433019,2391791,202557,11,6,6,8.0,43553,2,1,Orange Energy Shots,...,3.7,low-range product,regular days,fewest orders,31,regular customer,6.905655,low spender,8.0,frequent customer


In [10]:
# finding 
(opm
.groupby('loyalty_flag')
.agg({'prices':['mean','max','min']})
)

prices           
                      mean   max  min
loyalty_flag                         
loyal customer    7.778434  25.0  1.0
new customer      7.805661  25.0  1.0
regular customer  7.802976  25.0  1.0

In [4]:
# creating a new col with average prices
opm['average_price'] = (opm
                    .groupby(['user_id'])['prices']
                    .transform(np.mean))

In [5]:
# checking to see if calculation was done correctly
opm[opm['user_id']==203436]['prices'].mean()

7.631578947368421

In [6]:
opm.loc[opm['user_id'] == 203436,['user_id','prices','average_price']]

,user_id,prices,average_price
2211676,203436,4.0,7.631579
2211677,203436,4.0,7.631579
3673339,203436,7.3,7.631579
4090301,203436,3.8,7.631579
4159638,203436,7.5,7.631579
4276527,203436,6.3,7.631579
4276528,203436,6.3,7.631579
4900856,203436,7.9,7.631579
5375004,203436,14.3,7.631579
5375005,203436,14.3,7.631579


In [7]:
# creating a new col to flag customers

opm.loc[opm['average_price'] < 10, 'spender_type'] = 'low spender'
opm.loc[opm['average_price'] >= 10, 'spender_type'] = 'high spender'

In [11]:
# creating median days since ordered col per user

opm['median_day_since_ordered'] = (opm
                    .groupby(['user_id'])['days_since_prior_order']
                    .transform(np.median))

In [12]:
# checking to see if calc were done correctly
opm[opm['user_id']==202557]['days_since_prior_order'].median()

8.0

In [13]:
# creating a new col to flag frequent customers
opm.loc[opm['median_day_since_ordered'] <= 10, 'customer_freq'] = 'frequent customer'
opm.loc[(opm['median_day_since_ordered'] > 10) & (opm['median_day_since_ordered'] <=20), 'customer_freq'] = 'regular customer'
opm.loc[opm['median_day_since_ordered'] > 20, 'customer_freq'] = 'non-frequent customer'

In [14]:
# exporting data
opm.to_pickle(os.path.join(path,'opm.pkl'))